The following code queries all bakers in the 6 previous cycles and their versions.

To run, press Crtl + Enter three times.

In [ ]:
import requests
import json
import datetime
# get current cycle
current_cycle = int(requests.get("https://api.tzstats.com/explorer/cycle/head").json()["cycle"])

print("Starting queries... it will take a while")
# make list of active bakers
bakers_dict = {}
for cycle in range(current_cycle - 5, current_cycle+1):
    bakers_json = requests.get(f"https://api.tzstats.com/tables/income?&columns=address,rolls&cycle={cycle}")
    bakers = bakers_json.json()
    for baker in bakers:
        if baker[0] not in bakers_dict:
            bakers_dict[baker[0]] = { "rolls": baker[1] }

# uncomment line below to make the list smaller while developing
#bakers_dict = { k:v for k,v in bakers_dict.items() if k in list(bakers_dict)[:100]}

# query baker alias from tzkt (tzstats does not have it)

tzkt_account_json = requests.get("https://api.tzkt.io/v1/delegates/?select=address,alias,balance&limit=10000")
bakers_alias={account['address']:account['alias'] for account in tzkt_account_json.json()}

# hardcoded version numbers ... not sure where to look
version_nums = { "d74e7b6d":"v8.1",
               "6102c808": "v8.2",
               "ce365456": "v7.5",
               "e69b63f1": "v7.4",
               "567be003": "v8.0",
               "c90787b9": "v8.0-rc1",
               "13067b95": "mainnet-2020-june"}

for baker_idx, baker in enumerate(list(bakers_dict)):
    baker_version = requests.get(f"https://api.tzstats.com/explorer/account/{baker}").json()["baker_version"]
    # query baker alias from tzkt (tzstats does not have it)

    if baker_idx % 10 == 0:
        print(f"{len(bakers_dict) - baker_idx} bakers left to query...")
    bakers_dict[baker]["version"] = version_nums.get(baker_version, baker_version)
    bakers_dict[baker]["alias"] = bakers_alias.get(baker)
    
#Organize upgrade results in a table compatible with Pandas DataFrame
results_pn = {'address':[],
              'alias': [],
              'rolls':[],
              'version':[]
             }

for k,v in bakers_dict.items():
    results_pn['address'].append(k)
    results_pn['alias'].append(v['alias'])
    results_pn['rolls'].append(v['rolls'])
    results_pn['version'].append(v['version'])
    
print("all done!")

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', None)
df = pd.DataFrame(results_pn)
df = df.sort_values(by=['rolls'], ascending=False)
from IPython.display import display, HTML

display(df)